In [1]:
import pandas as pd
import numpy as np

In [2]:
df_icu = pd.read_csv('D:\Paper\Chart_Lab_by_hours\Total_file\All_icu_patients.csv')

df_admissions = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\core\Admissions.csv.gz', usecols = ['subject_id', 'hadm_id', 'ethnicity'])

df_patients = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\core\patients.csv.gz')

In [3]:
df_icu['date']= pd.to_datetime(df_icu['date'])

df_icu['year'] = df_icu['date'].dt.year

In [4]:
"""
Δημιουργία ξεχωριστά όλων των οχτάωρων αρχείων
με τις καταγραφές των ασθενών που περιλαμβάνουν
ηλικία, φύλλο, εθνικότητα.
"""

for i in range (1, (df_icu.eight_hour.max()+1)):
    # Επιλογή οχταώρου
    temp = df_icu.loc[df_icu['eight_hour'] == i]
    # Στο ενδεχόμενο να υπάρχουν διπλές εγγραφές
    final = temp.drop_duplicates(subset=['subject_id'])
    # ενοποίηση του οχταώρου με φύλλο, χρονιές
    rslt_df=pd.merge(final, df_patients, on=['subject_id'], how='left')
    # ενοποίηση με εθνικότητα
    rslt_df=pd.merge(rslt_df, df_admissions, on=['subject_id', 'hadm_id'], how='left')
    # υπολογισμός ηλικίας
    rslt_df['temp_age'] = rslt_df['year'] - rslt_df['anchor_year']
    # υπολογισμός ηλικίας
    rslt_df['Age'] = rslt_df['temp_age'] + rslt_df['anchor_age']
    # αφαίρεση μη χρήσιμων στηλών
    rslt_df = rslt_df.drop(['year', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod', 'temp_age' ], axis=1)
    # εξαγωγή csv
    rslt_df.to_csv('D:\Paper\Chart_Lab_by_hours\Test\o{}.csv'.format(i), index=False) 

In [5]:
"""
Ενοποίηση των οχτάωρων αρχείων με τις καταγραφές
των ασθενών που περιλαμβάνουν ηλικία, φύλλο, εθνικότητα.
"""

temp = pd.read_csv('D:\Paper\Chart_Lab_by_hours\Test\o1.csv')
for i in range (1, (df_icu.eight_hour.max())):
    tempII = (temp, pd.read_csv('D:\Paper\Chart_Lab_by_hours\Test\o{}.csv'.format(i+1)))             
    temp = pd.concat(tempII) 

In [6]:
for i in range (1, 4):
    cols = list(temp.columns)
    cols = [cols[-1]] + cols[:-1]
    temp = temp[cols]

# Στοίχιση πίνακα
temp.sort_values(['subject_id', 'hadm_id', 'date', 'time']).to_csv('D:\Paper\Chart_Lab_by_hours\Test\Merge_file.csv', index=False)